In [8]:
!pip install -q sentence-transformers faiss-cpu accelerate bitsandbytes datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 11.9 MB/s eta 0:00:00


In [9]:
from sentence_transformers import SentenceTransformer

ST = SentenceTransformer("ai-forever/sbert_large_nlu_ru")

In [10]:
from datasets import Dataset

text = """Студия дизайна интерьера Проектное Бюро 52

Вы здесь и это значит, что первый шаг к созданию великолепного, уютного, удобного, интуитивно понятного интерьера уже сделан!
Мы занимаемся разработкой проектов квартир, частных домов, коттеджей, офисных пространств, апартаментов, объектов HoReCa: отелей, баров и ресторанов.

Мы осуществляем реализацию дизайн интерьера в Нижнем Новгороде, предоставляя нашим клиентам полный цикл услуг от задумки до воплощения — проектирование, авторский надзор, комплектация мебелью, светом, сантехникой, декоративными материалами. Наши работы — это интерьеры, в которых хочется отдыхать, работать, жить. Нам интересны любые смелые и новые задачи.

На сайте мы постарались предоставить как можно больше информации по нашей деятельности, а также портфолио и контакты. Если остались вопросы — просто свяжитесь с нами!

С уважением Елена и Екатерина.


Стоимость дизайна интерьера в Нижнем Новгороде:
ЭКОНОМ
690 ₽
за кв.м
Пример проекта
Выезд на обмеры
Составление ТЗ
Варианты планировок
3D эскиз
Чертежи
Ведомость отделочных материалов, освещения, схемы заказной мебели
10 рабочих дней*
Минимальная стоимость 34 500 руб.


ЭСКИЗНЫЙ
1400 ₽
за кв.м
Пример проекта
Выезд на обмеры
Составление ТЗ
Варианты планировок
3D эскиз
Минимальный набор чертежей
Ведомость отделочных материалов, освещения, схемы заказной мебели
1 месяц*
Минимальная стоимость 70 000 руб.


СТАНДАРТ
1800 ₽
за кв.м
Пример проекта
Выезд на обмеры
Составление ТЗ
Варианты планировок
3D эскиз
Чертежи
Ведомость отделочных материалов, освещения, схемы заказной мебели
1-2 месяца*
Минимальная стоимость 90 000 руб.


СТАНДАРТ +
2500 ₽
за кв.м
Пример проекта
Выезд на обмеры
Составление ТЗ
Варианты планировок
3D фотореалистичная визуализация
Чертежи
Ведомость отделочных материалов, освещения, схемы заказной мебели
от 3-х месяцев*
Минимальная стоимость 125 000 руб.


Подбор – 400 руб. за кв.м. Минимальная стоимость 20 000 руб.
Подбор отделочных материалов, освещения, сантехники, мебели, текстиля и т.д
В стоимость входит не более 10 выездов в магазины, салоны, шоу-румы.
Возможен подбор онлайн в виде фото и видеоматериалов.

Авторский надзор – 300 руб. за кв.м в месяц. Минимальная стоимость 20 000 руб. за месяц.
Регулярный выезд на объект, с целью контроля работы за соответствием дизайн-проекту. При необходимости внесение корректировок в рабочие схемы.

“Просто прикинуть планировку” без обмеров – 200 руб. кв.м
Разовый выезд на строительную площадку или консультация – 6 000 руб. (не более 2х часов)

* Сроки указаны ориентировочно.

Цены актуальны для Нижнего Новгорода."""

dataset = Dataset.from_dict({"text": [text]})

def embed(batch):
    """
    adds a column to the dataset called 'embeddings'
    """
    # or you can combine multiple columns here
    # For example the title and the text
    information = batch["text"]
    return {"embeddings" : ST.encode(information)}

dataset = dataset.map(embed,batched=True,batch_size=16)

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [11]:
data = dataset.add_faiss_index("embeddings") # column name that has the embeddings of the dataset

  0%|          | 0/1 [00:00<?, ?it/s]

In [12]:
def search(query: str, k: int = 3 ):
    """a function that embeds a new query and returns the most probable results"""
    embedded_query = ST.encode(query) # embed new query
    scores, retrieved_examples = data.get_nearest_examples( # retrieve results
        "embeddings", embedded_query, # compare our new embedded query with the dataset embeddings
        k=k # get only top k results
    )
    return scores, retrieved_examples

In [13]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

model_id = "IlyaGusev/saiga_llama3_8b"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    quantization_config=bnb_config
)
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/277 [00:00<?, ?B/s]

In [14]:
SYS_PROMPT = """Ты - помощник, отвечающий на вопросы клиентов о компании.
Тебе даны вырезанные части длинного документа - базы знаний компании, и вопрос. Ты должен дать ответ на вопрос в разговорной форме.
Если ты не знаешь ответа, просто скажите «я не знаю». Не придумывайте ответ."""

In [15]:
def format_prompt(prompt,retrieved_documents,k):
  """using the retrieved documents we will prompt the model to generate our responses"""
  PROMPT = f"Вопрос:{prompt}\nКонтекст:"
  for idx in range(k) :
    PROMPT+= f"{retrieved_documents['text'][idx]}\n"
  return PROMPT

def generate(formatted_prompt):
  formatted_prompt = formatted_prompt[:2000] # to avoid GPU OOM
  messages = [{"role":"system","content":SYS_PROMPT},{"role":"user","content":formatted_prompt}]
  # tell the model to generate
  input_ids = tokenizer.apply_chat_template(
      messages,
      add_generation_prompt=True,
      return_tensors="pt"
  ).to(model.device)
  outputs = model.generate(
      input_ids,
      max_new_tokens=1024,
      eos_token_id=terminators,
      do_sample=True,
      temperature=0.6,
      top_p=0.9,
  )
  response = outputs[0][input_ids.shape[-1]:]
  return tokenizer.decode(response, skip_special_tokens=True)

def rag_chatbot(prompt:str,k:int=2):
  scores , retrieved_documents = search(prompt, k)
  formatted_prompt = format_prompt(prompt,retrieved_documents,k)
  return generate(formatted_prompt)

In [19]:
rag_chatbot("Дай мне прайс-лист ваших услуг", k=1)

'Конечно, я могу помочь вам с этим! Мы предлагаем несколько вариантов дизайна интерьера в зависимости от вашего бюджта и предпочтений. Стоимость наших услуг варьируется от экономного пакета (690 рублей за квадратный метр) до премиального (2500 рублей за квадратный метр). В каждом из этих пакетов включены различные услуги, такие как проектирование, авторский надзор, комплектация мебелью, светом и другими элементами декора. Также мы предлагаем подбор отделочных материалов для дополнительную плату.\n\nМинимальные стоимости наших услуг начинаются от 34 500 рублей для экономного пакета и достигают 125 000 рублей для премиального. Количество времени, которое потребуется для выполнения работ, варьируется от 10 рабочих дней для экономного пакета до нескольких месяцев для более дорогих вариантов.\n\nЕсли у вас есть конкретные идеи или желания, пожалуйста, не стесняйтесь обращаться ко мне, чтобы обсудить детали вашего проекта. Я буду рад помочь вам создать уникальный и функциональный интерьер, к